In [1]:
import numpy as np
import cv2
import torch
import torchvision
import os
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
os.chdir("/media/sowrya/DATA/My/Research/Faster-RCNN")
with torch.cuda.device(0):
    precision = 'fp32'
    ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)    
    utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')
    ssd_model.to('cuda')
    ssd_model.eval()
    labels = []
    with open("coco.names", "r") as f:
        for row in f:
            labels.append(str(row[:-1]))
    cwd = os.getcwd()
    dir_1 = cwd + "/New Test"
    dir_2 = cwd + "/Attacked"
    os.chdir(dir_1)
    names = os.listdir(dir_1)
    inputs = [utils.prepare_input(image) for image in names]
    tensor = utils.prepare_tensor(inputs, precision == 'fp16')
    with torch.no_grad():
        detections_batch = ssd_model(tensor)
    results_per_input = utils.decode_results(detections_batch)
    best_results_per_input = [utils.pick_best(results, 0.90) for results in results_per_input]
    classes_to_labels = utils.get_coco_object_dictionary()
    for image_idx in range(len(best_results_per_input)):
        fig, ax = plt.subplots(1)
        image = inputs[image_idx] / 2 + 0.5
        ax.imshow(image)
        bboxes, classes, confidences = best_results_per_input[image_idx]
        for idx in range(len(bboxes)):
            left, bot, right, top = bboxes[idx]
            x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    plt.show()
    os.chdir(dir_2)
    names = os.listdir(dir_2)
    inputs = [utils.prepare_input(image) for image in names]
    tensor = utils.prepare_tensor(inputs, precision == 'fp16')
    with torch.no_grad():
        detections_batch = ssd_model(tensor)
    results_per_input = utils.decode_results(detections_batch)
    best_results_per_input = [utils.pick_best(results, 0.90) for results in results_per_input]
    classes_to_labels = utils.get_coco_object_dictionary()
    for image_idx in range(len(best_results_per_input)):
        fig, ax = plt.subplots(1)
        image = inputs[image_idx] / 2 + 0.5
        ax.imshow(image)
        bboxes, classes, confidences = best_results_per_input[image_idx]
        for idx in range(len(bboxes)):
            left, bot, right, top = bboxes[idx]
            x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)
            ax.text(x, y, "{} {:.0f}%".format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
    plt.show()

Using cache found in /home/sowrya/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


HTTPError: HTTP Error 404: 